In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer
from transformers import AutoTokenizer, AutoModelWithLMHead

import torch
from transformers import BertForSequenceClassification, Trainer, TrainingArguments

## 데이터 로드

###### 디씨

In [2]:
dc1 = pd.read_csv('./dc_filtered1.csv')
dc2 = pd.read_csv('./dc_filtered_cho.csv')
dc3 = pd.read_csv('./dc_filtered_Choi.csv')

In [3]:
dc_merged = pd.merge(pd.merge(dc1, dc2, on='comments', how='outer'), dc3, on='comments', how='outer')

In [4]:
dc_merged['hate'] = (dc_merged[['hate1', 'hate2', 'hate3']].sum(axis=1) >= 2).astype(int)
dc_merged = dc_merged[['comments', 'hate']]
dc_merged.head()

,comments,hate
0,느그집에 장애인이있나봐? ㅋㅋㅋㅋ,1
1,애미애비 둘 중 하나가 장애인인 병신,1
2,장애인 한마리 부들부들거리노,1
3,딸따리~~,0
4,ㄴ 모르면 닥쳐 정신병자 새끼야 ㅋ,1


###### 네이버

In [5]:
nc1 = pd.read_csv('./ndata_merged1.csv')
nc2 = pd.read_csv('./ndata_merged_cho.csv')
nc3 = pd.read_csv('./ndata_merged_Choi.csv')

In [6]:
nc_merged = pd.concat([pd.concat([nc1, nc2['hate2']], axis = 1), nc3['hate3']], axis = 1)

In [7]:
nc_merged['hate'] = (nc_merged[['hate', 'hate2', 'hate3']].sum(axis=1) >= 2).astype(int)
nc_merged = nc_merged[['comments', 'hate']]
nc_merged.head()

,comments,hate
0,하바드 하바드 하니 마국 ADA 법도 알겠네 \n\n거기 보면 장애인도 모든 대...,0
1,장애인 단체의 시위 방법이 온당하다고는 할 수 없지만 사회의 갈등을 해소해야 할...,0
2,진짜로 장애인의 권익 증진이 목적이 아닌거 같으니까 문제지. 장애인 좌익 농성단체 ...,0
3,정상적으로 시위하면 누가 뭐라고 하나.... 장애인들은 점거하고 문 막고 그렇게 해...,0
4,이건 아닌거 같은데..전장연이 큰걸 바란게 아닌거 같은데 과하게 대응한다.,0


In [62]:
nc_merged[:20]

,comments,hate
0,하바드 하바드 하니 마국 ADA 법도 알겠네 \n\n거기 보면 장애인도 모든 대...,0
1,장애인 단체의 시위 방법이 온당하다고는 할 수 없지만 사회의 갈등을 해소해야 할...,0
2,진짜로 장애인의 권익 증진이 목적이 아닌거 같으니까 문제지. 장애인 좌익 농성단체 ...,0
3,정상적으로 시위하면 누가 뭐라고 하나.... 장애인들은 점거하고 문 막고 그렇게 해...,0
4,이건 아닌거 같은데..전장연이 큰걸 바란게 아닌거 같은데 과하게 대응한다.,0
5,"이준석 말이 옳다, 이대로 장애인이 시민의 발을 불모로 잡는 다면 민주당은 지방선거...",0
6,그동안 장애인들 목소리에 얼마나 귀 기울였다고... 누군가에겐 불편함이지만 누군가에...,0
7,장애인 = 착하다. 라는 선입견부터 버려???한다.,0
8,"이준석 대표 틀린 말 없는데 이준석 대표 전폭지지 장애인, 비장애인, 전교조, ...",0
9,장애인에겐 평등을 똑같이 해주면됨 미쳐날뛰는 이유는 엄청난 혜택때문임,1


###### chat GPT 생성

In [8]:
gpt = pd.read_csv('ChatGPT 생성 댓글.txt', header=None, names=['comments'], sep = '\n')
gpt[['hate']]=1

In [9]:
gpt_neutral = pd.read_csv('ChatGPT_neutral.txt', header=None, names=['comments'], sep = '\n')
gpt_neutral[['hate']]=0

In [10]:
gpt_neutral = gpt_neutral.drop_duplicates(subset='comments')

In [11]:
gpt = pd.concat([gpt, gpt_neutral], ignore_index = True)
gpt.head()

,comments,hate
0,장애인들 특혜 받으려고 인권 운운하는 거지.,1
1,장애자들 인권만 중요하고 남 인권은 안 보이나?,1
2,애자들 혜택 그만 좀 요구해라.,1
3,장애자들 인권 운운하면서 일반인 인권 침해하는 거 안 보이냐?,1
4,병신 새끼들이라고 다 봐줘야 하는 거냐?,1


In [12]:
len(gpt)

301

###### 위키백과

In [13]:
#https://ko.wikipedia.org/wiki/장애인
# 장애인 인권/발달장애/학습장애/자폐/청각_장애/신체_장애/지적_장애/전반적_발달장애
import re

# 1. 'wiki' 텍스트 파일 읽어오기
wiki_txt = open('wiki.txt', 'r', encoding='utf-8').read()

# 2. 문장 단위 분리 및 ['숫자'] 형식의 글자 제거
wiki_sents = [re.sub(r'\[\d+\]', '', sentence).strip() for sentence in wiki_txt.split('.')]

# 3. 데이터프레임 생성
wiki = {
    'comments': wiki_sents,
    'hate': [0] * len(wiki_sents)
}
wiki = pd.DataFrame(wiki)


In [71]:
wiki

,comments,hate
0,"장애인(障礙人)은 신체적, 정신적 손상 등으로 인한 사회적 차별로 인해 일상생활에 ...",0
1,장애인들은 역사적으로도 비참한 인생과 최후를 맞을 가능성이 있었으며 특히 지적 능력...,0
2,남자의 경우도 지적장애인들이 섬노예로 장기간 고통을 받았던 역사가 있다,0
3,"장애의 유래에 대한 설명은 장애의 의학적 모델과 장애의 사회적 모델이 있는데, 현재...",0
4,"크게 장애는 신체적 장애와 정신적 장애로 나뉘는데, 신체적 장애의 경우 태어났을 때...",0
...,...,...
491,아동에 대한 치료는 아동의 특정한 필요에 따라 특정화되어야 한다,0
492,일부 아동은 규모가 작고 1:1 교육을 수행하는 특수화 교실에서 효과가 크다,0
493,다른 아동은 표준 특별교육 수업이나 지원이 들어가는 정규 수업에서 잘 기능한다,0
494,"절절하고 특수화된 교육 프로그램과 지원 서비스 등, 초기 개입은 환자의 결과를 좋게...",0


###### 합치기

In [14]:
hate = pd.concat([nc_merged, dc_merged, wiki, gpt], ignore_index=True)

In [15]:
hate = hate.sample(frac=1, random_state=42).reset_index(drop=True)

In [16]:
hate.to_csv('hate_data.csv')

In [17]:
len(hate)

2294

## KcBert-base

In [18]:
hate.hate.sum()

813

In [19]:
comments = hate['comments'].tolist()
labels = hate['hate'].tolist()

#학습 & 검증 데이터셋 분리
train_texts, eval_texts, train_labels, eval_labels = train_test_split(comments, labels, test_size=0.2, random_state=42)

In [20]:
##KcBERT 로드
from transformers import AutoTokenizer, AutoModelWithLMHead
tokenizer = AutoTokenizer.from_pretrained("beomi/kcbert-base")

In [21]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

In [22]:
def tokenize(texts, labels):
    if isinstance(texts, pd.Series):
        texts = texts.tolist()
    elif not isinstance(texts, list):
        raise ValueError("texts should be a list of strings")
    
    encodings = tokenizer(texts, padding=True, truncation=True, return_tensors="pt")
    return encodings, torch.tensor(labels)  # Convert labels to a tensor


In [23]:
# Example assuming a straightforward approach
train_texts = hate['comments'].astype(str).tolist()  
train_labels = hate['hate'].tolist()  

eval_texts = hate['comments'].astype(str).tolist() 
eval_labels = hate['hate'].tolist()  

# 토큰화
train_encodings, train_labels = tokenize(train_texts, train_labels)
eval_encodings, eval_labels = tokenize(eval_texts, eval_labels)

In [24]:
import torch
from torch.utils.data import Dataset

class HateSpeechDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx], dtype=torch.long)
        return item

# PyTorch 데이터셋 생성
train_dataset = HateSpeechDataset(train_encodings, train_labels)
eval_dataset = HateSpeechDataset(eval_encodings, eval_labels)

In [25]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

# KcBERT model 로드
model = AutoModelForSequenceClassification.from_pretrained("beomi/kcbert-base", num_labels=2)

# Set up training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=4,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_dir='./logs',
    logging_steps=10,
)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
)

# Train the model
trainer.train()

# 모델 저장
model.save_pretrained('./kcbert_base_abelism_model')


Some weights of the model checkpoint at beomi/kcbert-base were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at beomi/kcbert-base and are newly i

Epoch,Training Loss,Validation Loss
1,0.321900,0.153171
2,0.164100,0.040717
3,0.068700,0.013758
4,0.013200,0.005693


c:\users\user\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  
c:\users\user\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  
c:\users\user\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  


In [26]:
# 학습된 모델 및 토크나이저 로드
trained_model = AutoModelForSequenceClassification.from_pretrained('./kcbert_base_abelism_model')
trained_tokenizer = AutoTokenizer.from_pretrained("beomi/kcbert-base")


In [27]:
def predict(text):
    inputs = trained_tokenizer(text, return_tensors="pt", padding=True, truncation=True)
    outputs = trained_model(**inputs)
    probabilities = torch.nn.functional.softmax(outputs.logits, dim=-1)
    # Extract the probability of the positive class
    score = probabilities.detach().numpy()[:, 1][0]  # Gets the second element from the first item in batch
    # Return a dictionary with the text and its associated score
    return {text: score}


In [29]:
print(predict("ㅄ이니?"))
print(predict("페미들이랑 똑같노 능지 수준 알 만하다"))
print(predict("장애인들이 세금을 좀먹는다"))
print(predict('장애인은 신체적, 정신적 장애를 가진 사람들이다.'))
print(predict("몸 병신인 줄 알았는데 머리도 병신이었니?"))
print(predict("장애는 질병이다"))
print(predict("장애인들은 장애를 가진 사람들이다"))
print(predict('너 능지가 처참한 수준이구나'))
print(predict('너새끼 장애니?'))
print(predict('다리 병신인줄 알았는데 팔도 병신'))
print(predict('어 저기 휠룡인 지나간다 ㅋㅋㅋ'))
print(predict('장애인은 장애 관련 법령에 의거하여 보호를 받는다.'))

{'ㅄ이니?': 0.00039990005}
{'페미들이랑 똑같노 능지 수준 알 만하다': 0.006539432}
{'장애인들이 세금을 좀먹는다': 0.99950755}
{'장애인은 신체적, 정신적 장애를 가진 사람들이다.': 0.00011990118}
{'몸 병신인 줄 알았는데 머리도 병신이었니?': 0.99924076}
{'장애는 질병이다': 0.9978224}
{'장애인들은 장애를 가진 사람들이다': 0.008893965}
{'너 능지가 처참한 수준이구나': 0.00417608}
{'너새끼 장애니?': 0.9992199}
{'다리 병신인줄 알았는데 팔도 병신': 0.99830073}
{'어 저기 휠룡인 지나간다 ㅋㅋㅋ': 0.9953491}
{'장애인은 장애 관련 법령에 의거하여 보호를 받는다.': 6.9341084e-05}


In [192]:
print(predict(''))

{'장애인은 정상인이다': 0.9994616}


# KcBERT-large

In [206]:
comments = hate['comments'].tolist()
labels = hate['hate'].tolist()

#학습 & 검증 데이터셋 분리
train_texts, eval_texts, train_labels, eval_labels = train_test_split(comments, labels, test_size=0.2, random_state=42)

In [207]:
from transformers import AutoTokenizer, AutoModelWithLMHead

In [208]:
# Large Model (334M)
tokenizer = AutoTokenizer.from_pretrained("beomi/kcbert-large")

def tokenize(texts, labels):
    if isinstance(texts, pd.Series):
        texts = texts.tolist()
    elif not isinstance(texts, list):
        raise ValueError("texts should be a list of strings")
    
    encodings = tokenizer(texts, padding=True, truncation=True, return_tensors="pt")
    return encodings, torch.tensor(labels)  # Convert labels to a tensor


In [209]:
# Example assuming a straightforward approach
train_texts = hate['comments'].astype(str).tolist()  
train_labels = hate['hate'].tolist()  

eval_texts = hate['comments'].astype(str).tolist() 
eval_labels = hate['hate'].tolist()  

In [210]:
# 토큰화
train_encodings, train_labels = tokenize(train_texts, train_labels)
eval_encodings, eval_labels = tokenize(eval_texts, eval_labels)

In [211]:
import torch
from torch.utils.data import Dataset

class HateSpeechDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx], dtype=torch.long)
        return item

# PyTorch 데이터셋 생성
train_dataset = HateSpeechDataset(train_encodings, train_labels)
eval_dataset = HateSpeechDataset(eval_encodings, eval_labels)

In [212]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

# KcBERT model 로드
model = AutoModelForSequenceClassification.from_pretrained("beomi/kcbert-large", num_labels=2)

# Set up training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_dir='./logs',
    logging_steps=10,
)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
)

# Train the model
trainer.train()

# 모델 저장
model.save_pretrained('./kcbert_large_abelism_model')


Some weights of the model checkpoint at beomi/kcbert-large were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at beomi/kcbert-large and are newly

Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 

# KcELECTRA

In [234]:
comments = hate['comments'].tolist()
labels = hate['hate'].tolist()

# Split into training and evaluation datasets
train_texts, eval_texts, train_labels, eval_labels = train_test_split(comments, labels, test_size=0.2, random_state=42)

In [236]:
tokenizer = AutoTokenizer.from_pretrained("beomi/KcELECTRA-base")

# Tokenization helper function
def tokenize(texts, labels):
    return tokenizer(texts, padding=True, truncation=True, return_tensors="pt"), labels

# Tokenize datasets
train_encodings, train_labels = tokenize(train_texts, train_labels)
eval_encodings, eval_labels = tokenize(eval_texts, eval_labels)

In [237]:
class HateSpeechDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx], dtype=torch.long)
        return item

# Create PyTorch datasets
train_dataset = HateSpeechDataset(train_encodings, train_labels)
eval_dataset = HateSpeechDataset(eval_encodings, eval_labels)

In [238]:
model = AutoModelForSequenceClassification.from_pretrained("beomi/KcELECTRA-base", num_labels=2)

# Set up training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_dir='./logs',
    logging_steps=10,
)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
)

# Train the model
trainer.train()

# Save the fine-tuned model
model.save_pretrained('./kcelectra_hate_speech_model')

Some weights of the model checkpoint at beomi/KcELECTRA-base were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense.weight', 'discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.weight']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at beomi/KcELECTRA-base and are newly initialized: ['classifier.out_proj.bias', 'classifier

Epoch,Training Loss,Validation Loss
1,0.348400,0.390590
2,0.231800,0.423672
3,0.203900,0.436517


In [241]:
# Load the trained model and tokenizer
trained_model = AutoModelForSequenceClassification.from_pretrained('./kcelectra_hate_speech_model')
trained_tokenizer = AutoTokenizer.from_pretrained("beomi/KcELECTRA-base")

# Predict function
def predict(text):
    inputs = trained_tokenizer(text, return_tensors="pt", padding=True, truncation=True)
    outputs = trained_model(**inputs)
    probabilities = torch.nn.functional.softmax(outputs.logits, dim=-1)
    return probabilities.detach().numpy()


[[0.03469825 0.9653017 ]]


In [247]:

# Example usage
example_comment = "그 사람 장애인이니?"
predicted_score = predict(example_comment)
print(predicted_score)


[[0.03198374 0.9680162 ]]
